In [ ]:
!pip install azure-storage-blob
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine
import psycopg2
from datetime import datetime

In [ ]:
URL = "https://data.cityofnewyork.us/resource/upii-frjc.csv?$limit=15000"

df_raw=pd.read_csv(URL, engine = 'pyarrow')
df_raw.head()


,violation_number,bic_number,account_name,type_of_violation,violation_phase,violation_disposition,account_type,violation_account_address,violation_account_city,violation_account_state,...,description_of_rule,export_date,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,TWC-225833,501970.0,COGENT WASTE SOLUTIONS LLC,OATH,Issued,Settled,License,860 HUMBOLDT STREET,BROOKLYN,NY,...,A licensee must maintain copies of all daily i...,2024-05-14,40.730750,-73.946279,1.0,33.0,579.0,3392243.0,3.025820e+09,Greenpoint
1,TWC-218787,NaN,NOTIAS CONSTRUCTION INC.,OATH,Issued,Settled,Unlicensed,29-10 120TH STREET,FLUSHING,NY,...,Failed to provide off-street parking for vehic...,2024-05-14,40.771853,-73.847965,7.0,19.0,907.0,4539993.0,4.043460e+09,College Point
2,TWC-213233,NaN,SOTERO MONGE,OATH,Issued,Settled,Unlicensed,19 SCHOOL STREET,INWOOD,NY,...,Failure to comply with a Commission Directive,2024-05-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,TWC-212090,3578.0,"CITYWIDE GENERAL CONSTRUCTION, INC.",OATH,Issued,Settled,CL2 - Exempt,2911 BAISLEY AVENUE,BRONX,NY,...,Plates shall at all times be affixed in the ma...,2024-05-14,40.835595,-73.829338,10.0,13.0,184.0,2073159.0,2.053340e+09,Schuylerville-Throgs Neck-Edgewater Park
4,TWC-215336,4275.0,CROSS COUNTRY CONSTRUCTION LLC,OATH,Issued,Settled,CL2 - Exempt,3 WEST MAIN ST. #104,ELMSFORD,NY,...,Failure to provide information or documentatio...,2024-05-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [ ]:
df_raw.dropna()
df_raw.shape

(12861, 31)

In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12861 entries, 0 to 12860
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   violation_number            12861 non-null  object             
 1   bic_number                  6137 non-null   float64            
 2   account_name                12861 non-null  object             
 3   type_of_violation           12861 non-null  object             
 4   violation_phase             12861 non-null  object             
 5   violation_disposition       12861 non-null  object             
 6   account_type                12861 non-null  object             
 7   violation_account_address   12861 non-null  object             
 8   violation_account_city      12861 non-null  object             
 9   violation_account_state     12861 non-null  object             
 10  violation_account_postcode  12861 non-null  object        

In [ ]:
df_cleaned = df_raw.copy()

In [ ]:
df_cleaned.drop = df_raw.drop

In [ ]:
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [ ]:
# specify the path to your Json configuration file
config_file_path = 'config.json'


# load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)




CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'bic'
blob_name = 'bic2024_v2'


output = StringIO()
df_cleaned.to_csv(output, index = False)
data = output.getvalue()
output.close()

blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded bic2024_v2 to Azure Blob Storage in container bic.
